In [9]:
import pandas as pd
import numpy as np
from functools import partial
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from skopt import gp_minimize
from skopt import space

In [10]:
x=pd.read_csv('x_train.csv')
y=pd.read_csv('y_train.csv')


In [11]:
x.shape

(590540, 315)

In [12]:
X=x.values
Y=y.values.ravel()

In [13]:
classifier = ensemble.RandomForestClassifier(n_jobs=-1)

In [14]:
param_grid = {
 "n_estimators": np.arange(100, 1500, 100),
 "max_depth": np.arange(1, 31),
 "criterion": ["gini", "entropy"]
 }

In [15]:
model = model_selection.RandomizedSearchCV(
 estimator=classifier,
 param_distributions=param_grid,
 n_iter=5,
 scoring="accuracy",
 verbose=10,
 n_jobs=1,
 cv=3
 )

In [16]:
model.fit(X,Y)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV 1/3; 1/5] START criterion=entropy, max_depth=25, n_estimators=1200..........
[CV 1/3; 1/5] END criterion=entropy, max_depth=25, n_estimators=1200; total time= 7.8min
[CV 2/3; 1/5] START criterion=entropy, max_depth=25, n_estimators=1200..........
[CV 2/3; 1/5] END criterion=entropy, max_depth=25, n_estimators=1200; total time= 5.9min
[CV 3/3; 1/5] START criterion=entropy, max_depth=25, n_estimators=1200..........
[CV 3/3; 1/5] END criterion=entropy, max_depth=25, n_estimators=1200; total time= 5.0min
[CV 1/3; 2/5] START criterion=gini, max_depth=20, n_estimators=200..............
[CV 1/3; 2/5] END criterion=gini, max_depth=20, n_estimators=200; total time= 1.3min
[CV 2/3; 2/5] START criterion=gini, max_depth=20, n_estimators=200..............
[CV 2/3; 2/5] END criterion=gini, max_depth=20, n_estimators=200; total time= 1.1min
[CV 3/3; 2/5] START criterion=gini, max_depth=20, n_estimators=200..............
[CV 3/3; 2/5] END

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(n_jobs=-1), n_iter=5,
                   n_jobs=1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),
                                        'n_estimators': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100,
       1200, 1300, 1400])},
                   scoring='accuracy', verbose=10)

In [22]:
best_param=model.best_estimator_.get_params()
print(best_param)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': -1, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [26]:
df=pd.DataFrame.from_records([best_param])
df.head()
#new.to_csv('best_param.csv')

,bootstrap,ccp_alpha,class_weight,criterion,max_depth,max_features,max_leaf_nodes,max_samples,min_impurity_decrease,min_impurity_split,min_samples_leaf,min_samples_split,min_weight_fraction_leaf,n_estimators,n_jobs,oob_score,random_state,verbose,warm_start
0,True,0.0,None,gini,2,auto,None,None,0.0,None,1,2,0.0,200,-1,False,None,0,False


In [27]:
df.to_csv('best_param.csv')

In [28]:
model2=ensemble.RandomForestClassifier(**best_param)

In [29]:
model2.fit(X,Y)

RandomForestClassifier(max_depth=2, n_estimators=200, n_jobs=-1)

In [30]:
tst_out=pd.read_csv('ftst.csv')
tran_id=pd.read_csv('trans_id.csv')

In [31]:
to=tst_out.values

In [32]:
to[:5]

array([[ 2.2367987 ,  2.38925204, -0.53836314, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.23681043,  2.38926114, -0.35552079, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.23686322,  2.38930012, -0.40854754, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.23688669,  2.38931745, -0.53836314, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.23689255,  2.38932698, -0.3460795 , ...,  0.        ,
         1.        ,  0.        ]])

In [33]:
ti=tran_id.values


In [35]:
ti[:5]

array([[3663586],
       [3663588],
       [3663597],
       [3663601],
       [3663602]])

In [36]:
tp=model2.predict_proba(to)

In [37]:
tp[:5]

array([[0.92299909, 0.07700091],
       [0.9246643 , 0.0753357 ],
       [0.93201475, 0.06798525],
       [0.89974692, 0.10025308],
       [0.66002855, 0.33997145]])

In [39]:
tp.shape

(506691, 2)

In [40]:
to.shape

(506691, 315)

In [41]:
Y.shape

(590540,)

In [42]:
X.shape

(590540, 315)

In [43]:
model3=ensemble.RandomForestClassifier()
model3.fit(X,Y)

RandomForestClassifier()

In [46]:
res=model3.predict_proba(to)


In [48]:
res.shape

(506691, 2)

In [49]:
tp[:5,1]

array([0.07700091, 0.0753357 , 0.06798525, 0.10025308, 0.33997145])

In [50]:
tp[:5,0]

array([0.92299909, 0.9246643 , 0.93201475, 0.89974692, 0.66002855])

In [52]:
ti.shape

(506691, 1)

In [53]:
ti[:5]

array([[3663586],
       [3663588],
       [3663597],
       [3663601],
       [3663602]])

In [54]:
submission3 = pd.DataFrame({
    'TransactionID' : ti.ravel() ,
    'isFraud' : tp[:,1]
})

In [56]:
submission3.to_csv('subRFhpt.csv', index=False)

In [57]:
submission3.head()

,TransactionID,isFraud
0,3663586,0.077001
1,3663588,0.075336
2,3663597,0.067985
3,3663601,0.100253
4,3663602,0.339971
